# 経済指標

In [1]:
import pandas as pd
import datetime
import time
import re
import datetime
import numpy as np
import lxml.html

In [2]:
countries = [
    'アメリカ',
    'ドイツ',
    '日本',
    '英国',
    'ユーロ',
    'カナダ',
    'トルコ',
    '豪',
    '中国',
    'スイス',
    '仏',
    '南ア',
    '香港',
    'NZ',
    '韓国',
    'シンガポール',
    'ブラジル',
    'インド',
]

In [3]:
def get_df(url, date):
    url_date = url + date
    df = pd.read_html(url_date)
    df = df[0].dropna(subset = [4]).drop(2,axis =1)
    df.columns = ["発表時刻", "経済指標", "前回変動幅(USD/JPY)","前回","予想","結果"]
    df["日付"] = date
    return df

In [4]:
def str2num(text):
    '''
    表中の単位付きデータから数値を抜き出す
    データのカラムにapplyする
    かっこの中身を優先して読み取る。（速報値用）
    args:
        text: string or int or float
    return:
        num: float
    '''
    if((type(text) == int) or (type(text) == float)):
        return text
    if len(text.split('（')) == 2:
        text = text.split('（')[1]
    pattern = '[+]?[-]?[0-9]+[.]?[0-9]+?'
    result = re.match(pattern, text)
    if result == None:
        return 0
    return  float(text[result.start():result.end()])

In [5]:
def del_duplicate(df):
    '''
    データを古い順番に並べ替えて、重複データを削除する。
    新しい順だと新しいデータが残ってしまう。
    args:
        df: pd.DataFrame, columns = ['発表時刻', '経済指標', '前回変動幅(USD/JPY)', '前回', '予想', '結果', '日付']
    return:
        df: reversed, del duplicates
    '''
    df = df.reset_index(drop=True)
    cols = df.columns
    idx = list(df.index.values)
    idx.reverse()
    
    def inv_bool(a):
        if a == False:
            return True
        else:
            return False
    
    df_rev = df.iloc[idx].reset_index(drop=True)
    return df_rev[df_rev[cols[:-1]].duplicated().apply(inv_bool)].reset_index(drop=True)

In [6]:
def datetime_merge(d, t):
    dt = d + " " + t
    return datetime.datetime.strptime(dt, '%Y-%m-%d %H:%M')

In [7]:
def indicators(text):
    '''
    経済指標を成型して、指標を取り出す。
    '''
    if len(text.split('[')) > 1:
        return "-"
    return text.split(' ')[0]
    
def ind_filter(df, ind):
    '''
    指定した指標、国名のデータを取り出す。
    '''
    return df[df['経済指標'].str.contains(ind, na=False)]

In [8]:
# N = 820
# pdf = pd.DataFrame()
# for i in range(N):
#     date = datetime.date(2021, 4, 1) - datetime.timedelta(days=i)
#     date = date.strftime('%Y-%m-%d')
#     url = f'https://fx.minkabu.jp/indicators?date='
#     pdf_date = get_df(url, date)
#     pdf = pd.concat([pdf, pdf_date], axis=0)
#     time.sleep(0.1)
# pdf = pdf.reset_index(drop=True)

In [9]:
file_name = '../data/fx_indicators.csv' 
# pdf.to_csv(file_name, index=False)
pdf = pd.read_csv(file_name)
pdf = del_duplicate(pdf)
pdf.head()

,発表時刻,経済指標,前回変動幅(USD/JPY),前回,予想,結果,日付
0,22:30,アメリカ・新規失業保険申請件数 12/23 - 12/29,+0.6pips,21.6万件 （22.1万件）,22.0万件,23.1万件,2019-01-03
1,22:15,アメリカ・ADP雇用者数 12月,-1.2pips,17.9万人 （15.7万人）,18.0万人,27.1万人,2019-01-03
2,21:00,アメリカ・MBA住宅ローン申請指数 12/22 - 12/28,---,-1.4%,---,-8.5%,2019-01-03
3,17:30,[前年比],NaN,5.2% （5.3%）,4.6%,1.2%,2019-01-03
4,17:30,香港・小売売上高 11月 [前年比],-1.2pips,5.9% （6.0%）,4.5%,1.4%,2019-01-03


In [10]:
pdf['country'] = pdf['経済指標'].apply(lambda x: x.split("・")[0] if x.split("・")[0] in countries else '日本')
pdf['pips_reaction'] = pdf['前回変動幅(USD/JPY)'].apply(str2num).fillna(0)
pdf['datetime'] =  pdf.apply(lambda row:datetime_merge(*row[['日付', '発表時刻']]), axis=1)

pdf['previous_value'] = pdf['前回'].apply(str2num)
pdf['predicted_value'] = pdf['予想'].apply(str2num)
pdf['actual_value'] = pdf['結果'].apply(str2num)

pdf['indicators'] = pdf['経済指標'].apply(lambda x: x if x.split("・")[0] in countries else '日本・'+x)\
                .apply(lambda x: x.split("・")[1].split(' ')[0].split('[')[0])

In [11]:
cols = ['datetime', 'country', 'indicators', 'pips_reaction', 'previous_value', 'predicted_value', 'actual_value']
cond = pdf['indicators'] != ''
indicator_df = pdf[cond][cols].copy().reset_index(drop=True)

In [12]:
file_name = '../data/preprocessed_fx_indicators.csv' 
indicator_df.to_csv(file_name, index=False)